## Outils pour la manipulation d'images et librairies.


In [126]:
import PIL
from PIL import Image
import numpy as np
#import scipy as sp
import os
from math import log10, sqrt

def load(filename):
    toLoad= Image.open(filename)
    return np.asarray(toLoad)


def psnr(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

def dct2(a):
    return sp.fft.dct( sp.fft.dct( a, axis=0, norm='ortho' ), axis=1, norm='ortho' )

def idct2(a):
    return sp.fft.idct( sp.fft.idct( a, axis=0 , norm='ortho'), axis=1 , norm='ortho')



## Normalisation de l'image (YCbCr et padding)

Question 1 : Donner le code qui transforme une image RGB en une image YCbCr. Vous pourrez produire
une matrice pour chaque composante, pour pouvoir plus facilement les manipuler indépendamment. Vous
pouvez stocker les données YCbCr comme des entiers ou des flottants, mais vous expliquerez votre choix.

In [127]:
def RGB_en_YCbCR(image):
    image_finale = image.copy()
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            R = image[i, j, 0]
            G = image[i, j, 1]
            B = image[i, j, 2]
            image_finale[i, j, 0] = 0.299 * R + 0.587 * G + 0.114 * B
            image_finale[i, j, 1] = -0.1687 * R - 0.3313 * G + 0.5 * B + 128
            image_finale[i, j, 2] = 0.5 * R - 0.4187 * G - 0.0813 * B + 128
    return np.uint8(image_finale)


Question 2 : Donner le code qui transforme une image YCbCr en une image RGB. Attention, les valeurs des
canaux RGB doivent être un entier dans [0, 255] qui pourra être codé sur un octet. Appliquer successivement
la transformation RGB vers YCbCr puis YCbCr vers RGB et vérifier que vous obtenez l’image de départ.
Vous consulterez la documentation des fonctions de numpy clip, uint8 et mask qui pourraient vous être
utiles.

In [128]:
def YCbCR_en_RGB(image):
    image_finale = image.copy()
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            Y = image[i, j, 0]
            Cb = image[i, j, 1]
            Cr = image[i, j, 2]
            image_finale[i, j, 0] = Y + 1.402 * (Cr - 128)
            image_finale[i, j, 1] = Y - 0.34414 * (Cb - 128) - 0.71414 * (Cr - 128)
            image_finale[i, j, 2] = Y + 1.772 * (Cb - 128)
    return np.uint8(np.clip(image_finale, 0, 255))


In [129]:
def rgb2ycbcr(im):
    xform = np.array([[.299, .587, .114], [-.1687, -.3313, .5], [.5, -.4187, -.0813]])
    ycbcr = im.dot(xform.T)
    ycbcr[:,:,[1,2]] += 128
    return np.uint8(ycbcr)

def ycbcr2rgb(im):
    xform = np.array([[1, 0, 1.402], [1, -0.34414, -.71414], [1, 1.772, 0]])
    rgb = im.astype(np.float64)
    rgb[:,:,[1,2]] -= 128
    rgb = rgb.dot(xform.T)
    np.putmask(rgb, rgb > 255, 255)
    np.putmask(rgb, rgb < 0, 0)
    return np.uint8(rgb)

Question 3 : Donner la fonction qui réalise ce padding ainsi que celle qui l’élimine et vérifier que
l’application de ces deux transformations laissent l’image inchangée.

In [130]:
def padding(image):
    return image

## Découpage en blocs et compression

## Écriture dans un fichier

## Décompression

## Tests 

In [131]:
test = load("test.png")
testYCbCr = YCbCR_en_RGB(RGB_en_YCbCR(test))
Image.fromarray(test,'RGB').show()
Image.fromarray(testYCbCr,'RGB').show()
print(psnr(test, testYCbCr))


KeyboardInterrupt: 